# WeatherPy


In [1]:
 # Dependencies and Setup

# visualizations
import matplotlib.pyplot as plt
import seaborn as sns

#data science - standard library
import pandas as pd
import numpy as np

#requests
import requests
from pprint import pprint
import time

#regression
from scipy.stats import linregress
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "outputcities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

621

### Perform API Calls

In [4]:
#Lists
lats = []
longs = []
temps = []
feel_likes = []
humids = []
pressures = []
speeds = []
cloudiness = []
cities_found = []

for i in range(len(cities)):
    #get the city
    city = cities[i]
    
    #create the url
    units = "imperial"
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&units={units}&appid={weather_api_key}"
#     print(url)

    #initiate requests
    response = requests.get(url)
#     print(response.status_code)
    
    #error checking
    if response.status_code == 200:
        data = response.json()
#         pprint(data)
        
        try:
            #extraction of data
            lat = data["coord"]["lat"]
            long = data["coord"]["lon"]
            temp = data["main"]["temp"]
            feels_like = data["main"]["feels_like"]
            humidity = data["main"]["humidity"]
            pressure = data["main"]["pressure"]
            speed = data["wind"]["speed"]
            clouds = data["clouds"]["all"]
            city_name = data["name"]

            #save the data
            lats.append(lat)
            longs.append(long)
            temps.append(temp)
            feel_likes.append(feels_like)
            humids.append(humidity)
            pressures.append(pressure)
            speeds.append(speed)
            cloudiness.append(clouds)
            cities_found.append(city_name)
        except Exception as e:
            print(f"Through exception for city {city}: {e}")
    
    elif response.status_code == 404:
        print(f"Missing data in OpenWeatherAPI for {city}")
    else:
        print(response.status_code)
        print("Oh gosh darnit. The API is broken. Sad Face.")
        
    #print every 5
    if (i % 5 == 0):
        print(f"Got data for city index: {i} of {len(cities)}")
    
    #sleep for requests
    time.sleep(1)
        
# make the dataframe
df = pd.DataFrame()
df["City"] = cities_found
df["Latitude"] = lats
df["Longitude"] = longs
df["Temperature"] = temps
df["Feels Like"] = feel_likes
df["Humidity"] = humids
df["Pressure"] = pressures
df["Wind Speed"] = speeds
df["Cloudiness"] = cloudiness

df.head(10)

Got data for city index: 0 of 621
Missing data in OpenWeatherAPI for taolanaro
Got data for city index: 5 of 621
Got data for city index: 10 of 621
Missing data in OpenWeatherAPI for bargal
Missing data in OpenWeatherAPI for illoqqortoormiut
Got data for city index: 15 of 621
Missing data in OpenWeatherAPI for barbar
Missing data in OpenWeatherAPI for amderma
Got data for city index: 20 of 621
Got data for city index: 25 of 621
Missing data in OpenWeatherAPI for banmo
Got data for city index: 30 of 621
Got data for city index: 35 of 621
Got data for city index: 40 of 621
Got data for city index: 45 of 621
Got data for city index: 50 of 621
Missing data in OpenWeatherAPI for tumannyy
Got data for city index: 55 of 621
Got data for city index: 60 of 621
Got data for city index: 65 of 621
Got data for city index: 70 of 621
Got data for city index: 75 of 621
Got data for city index: 80 of 621
Got data for city index: 85 of 621
Got data for city index: 90 of 621
Missing data in OpenWeatherA

,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness
0,Busselton,-33.65,115.33,59.00,51.91,80,996,14.16,95
1,Rikitea,-23.12,-134.97,73.13,68.92,76,1016,16.93,87
2,Clyde River,70.47,-68.59,-0.40,-17.95,70,1016,19.46,20
3,Hilo,19.73,-155.09,79.09,78.55,62,1017,10.29,75
4,Atuona,-9.80,-139.03,77.27,76.98,72,1011,11.92,40
5,Albany,42.60,-73.97,62.13,60.49,54,1024,0.94,0
6,Novopokrovka,49.84,36.55,44.31,38.80,93,1022,6.71,90
7,Nikolskoye,59.70,30.79,34.18,27.63,80,1022,4.47,20
8,Phetchabun,16.25,101.08,69.66,70.84,79,1014,5.66,0
9,Vaini,-21.20,-175.20,77.00,80.53,78,1015,6.93,20


### Convert Raw Data to DataFrame

In [5]:
df.to_csv(output_data_file, index=False)